For this exercise/project we will be using the `recipies.jsonl` file as our dataset. As for the `messages.jsonl` file you can use it to perform fine tuning directly in the console UI.

The source of this project is this project from OpenAI's cookbook:

https://cookbook.openai.com/examples/how_to_finetune_chat_models

The only modification we have done is in the dataset. I have provided you directly with the dataset to avoid some initial data preparation steps done in the cookbook project



# Setup

In [ ]:
# make sure to use the latest version of the openai python package
!pip install --upgrade --quiet openai

In [19]:
# Make sure to add OPENAI_API_KEY as your env var before running this code block
import openai
client = openai.OpenAI()

# Load Data

- Load and split the data into training and validation splits.
- The `recipies.jsonl` file has 122 examples which should be more than enough to perform the Project.
- We will use 80 examples for training and the rest 42 for validation purpose.


In [20]:
dataList = []
with open("recipies.jsonl") as f:
     dataList = [eval(line.rstrip('\n')) for line in f]

print(dataList[0])
print(type(dataList[0]))
print(len(dataList))

{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: Caprese Salad\n\nIngredients: ["3 ripe tomatoes, sliced", "1 cup fresh mozzarella cheese, sliced", "1/4 cup fresh basil leaves", "1/4 cup balsamic vinegar", "1/4 cup olive oil", "Salt and pepper to taste"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["tomatoes", "mozzarella cheese", "basil leaves", "balsamic vinegar", "olive oil", "salt", "pepper"]'}]}
<class 'dict'>
122


# Split Training and Validation Sets

In [21]:
trainingSet, validationSet = dataList[0:80], dataList[80:]

print(f"Training set has {len(trainingSet)} examples")
print(f"Validation set has {len(validationSet)} examples")

Training set has 80 examples
Validation set has 42 examples


# Upload data to OpenAI API

We then need to save our data as `.jsonl` files, with each line being one training example conversation.



In [22]:
import json

def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [23]:
training_file_name = "training_data.jsonl"
write_jsonl(trainingSet, training_file_name)

validation_file_name = "validation_data.jsonl"
write_jsonl(validationSet, validation_file_name)

In [24]:
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


training_file_id = upload_file(training_file_name, "fine-tune")
validation_file_id = upload_file(validation_file_name, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-HKSM9dANdGKLAl6WEjtOl51m
Validation file ID: file-OWsytGqjGpFAkU4O2woBZsb6


# Start Fine Tuning

In [33]:
MODEL = "gpt-3.5-turbo-0125"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    suffix="recipe-ner",
   hyperparameters={
      "n_epochs": 2
    }
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-wCfRaTBkkH6eU22hmjELFGzy
Status: validating_files


In [34]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-wCfRaTBkkH6eU22hmjELFGzy
Status: running
Trained Tokens: None


In [35]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-wCfRaTBkkH6eU22hmjELFGzy
Validating training file: file-HKSM9dANdGKLAl6WEjtOl51m and validation file: file-OWsytGqjGpFAkU4O2woBZsb6
Files validated, moving job to queued state
Fine-tuning job started


# Check Tuning Job Status

In [37]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError(
        "Fine-tuned model ID not found. Your job has likely not been completed yet."
    )

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:recipe-ner:9qrgclr6


# Perform Inference

In [39]:
test_messages = [
    {'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
    {'role': 'user', 'content': 'Title: Beef Brisket\n\nIngredients: ["4 lb. beef brisket", "1 c. catsup", "1 c. water", "1/2 onion, minced", "2 Tbsp. cider vinegar", "1 Tbsp. prepared horseradish", "1 Tbsp. prepared mustard", "1 tsp. salt", "1/2 tsp. pepper"]\n\nGeneric ingredients: '}
]

In [40]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

["beef brisket", "catsup", "water", "onion", "cider vinegar", "horseradish", "mustard", "salt", "pepper"]
